In [1]:
# imports
import numpy as np
import pandas as pd

In [2]:
# loading and visualizing dataset
dataset = pd.read_csv('/kaggle/input/cloud-coverage-detection/train.csv', low_memory=False)
dataset.head()

,DATE (MM/DD),MST,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Snow Depth [cm],Moisture,Albedo (CMP11)
0,01-Jan,00:00,-0.962276,0.000000,356.85640,7.216,0.988,-7.312,32.33,0.0,9.95,271.3,806.779,0.0,0.219,0.0,0.0
1,01-Jan,00:01,-0.937921,0.000000,357.65505,7.251,1.040,-7.260,32.40,0.0,8.20,272.9,806.840,0.0,0.206,0.0,0.0
2,01-Jan,00:02,-0.944395,0.000000,358.45438,7.256,1.093,-7.207,32.54,0.0,6.70,288.8,806.876,0.0,0.148,0.0,0.0
3,01-Jan,00:03,-0.951350,-0.029673,359.25416,7.254,1.060,-7.440,31.89,0.0,7.70,294.0,806.823,0.0,0.235,0.0,0.0
4,01-Jan,00:04,-0.934976,-0.054401,0.05415,7.331,1.081,-7.419,31.78,0.0,7.20,285.5,806.762,0.0,0.182,0.0,0.0


In [3]:
dataset.describe()

,Global CMP22 (vent/cor) [W/m^2],Direct sNIP [W/m^2],Azimuth Angle [degrees],Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Dew Point Temp [deg C],Tower RH [%],Total Cloud Cover [%],Peak Wind Speed @ 6ft [m/s],Avg Wind Direction @ 6ft [deg from N],Station Pressure [mBar],Precipitation (Accumulated) [mm],Snow Depth [cm],Moisture,Albedo (CMP11)
count,482400.000000,482400.000000,482400.000000,482400.000000,482400.000000,482400.000000,482400.000000,480931.000000,482400.000000,482400.000000,482400.000000,482400.000000,482400.000000,482400.000000,482400.000000
mean,203.417447,241.620827,180.002353,12.467373,5.067534,-2.698022,40.327665,23.611655,2.830681,168.190142,817.087196,0.374163,-2.826314,0.061655,0.129371
std,295.831990,369.333659,101.616425,10.578051,6.865294,7.315305,24.050161,34.740024,2.020588,116.355121,5.136720,1.928488,188.916827,0.237741,0.180258
min,-4.764200,-12.403300,0.000120,-16.690000,-17.351000,-24.830000,4.210000,0.000000,0.000000,0.000000,794.918000,0.000000,-7999.000000,0.000000,0.000000
25%,-1.081940,-0.197820,88.578010,4.170000,-0.261000,-8.109000,21.980000,0.000000,1.450000,44.580000,813.989000,0.000000,0.082000,0.000000,0.000000
50%,7.653785,0.266632,179.992330,13.020000,5.363000,-3.119000,33.630000,0.000000,2.450000,177.300000,817.553000,0.000000,0.656000,0.000000,0.000000
75%,351.550500,537.677250,271.428125,20.770000,11.096000,2.668000,53.010000,37.000000,3.700000,282.200000,820.553000,0.000000,1.032000,0.000000,0.201800
max,1428.650000,1069.200000,359.998560,36.320000,19.416000,17.418000,100.100000,100.000000,23.450000,360.000000,847.963000,28.190000,28.430000,1.000000,2.000000
